# 第11章：Agents基础（create_agent）

## 学习目标

本章将学习：
1. Agent的概念和架构
2. create_agent() API的使用
3. ReAct模式（Reasoning + Acting）
4. Agent的执行循环机制
5. system_prompt参数的作用
6. Agent基于LangGraph的底层架构
7. Agent vs 手动bind_tools的区别

## 什么是Agent？

**Agent = LLM + Tools + 自动循环**

Agent是能够自主决策和行动的AI系统：
- **感知**：接收用户输入
- **思考**：推理需要做什么
- **行动**：调用工具获取信息
- **循环**：基于结果继续思考，直到完成任务

## Agent的核心优势

1. **自主决策**：自己决定何时使用哪个工具
2. **自动循环**：无需手动管理工具调用循环
3. **错误恢复**：工具失败后可以重试或换策略
4. **多步推理**：可以执行复杂的多步骤任务

In [1]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_agent

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")

环境配置完成！
模型: gpt-4.1-mini


## 2. 创建第一个Agent

最简单的Agent：一个模型 + 一个工具

In [2]:
print("【创建简单Agent】")
print()

# 定义一个工具
@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气信息"""
    # 模拟天气数据
    weather_data = {
        "北京": "晴天，温度15°C",
        "上海": "多云，温度18°C",
        "深圳": "小雨，温度22°C"
    }
    return weather_data.get(city, f"{city}的天气信息暂时无法获取")

# 创建Agent
agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="你是一个天气助手，可以查询天气信息"
)

print("Agent创建成功！")
print(f"可用工具: {[t.name for t in [get_weather]]}")

【创建简单Agent】

Agent创建成功！
可用工具: ['get_weather']


### 2.1 运行Agent

In [3]:
print("【运行Agent】")
print()

# 调用Agent
response = agent.invoke({
    "messages": [{"role": "user", "content": "北京今天天气怎么样？"}]
})

# 获取最终回答
print("用户: 北京今天天气怎么样？")
print(f"Agent: {response['messages'][-1].content}")
print()
print(f"总共有 {len(response['messages'])} 条消息")

【运行Agent】

用户: 北京今天天气怎么样？
Agent: 北京今天天气晴朗，温度约为15°C。需要我帮你查询其他城市的天气吗？

总共有 4 条消息


## 3. Agent的执行循环

查看Agent内部的完整执行过程

In [6]:
print("【Agent执行循环详解】")
print()

# 重新运行，查看详细过程
response = agent.invoke({
    "messages": [{"role": "user", "content": "上海的天气如何？"}]
})

print("完整的消息历史：")
print("=" * 60)

for i, msg in enumerate(response['messages'], 1):
    msg_type = msg.__class__.__name__
    print(f"\n步骤 {i}: {msg_type}")    
    
    if hasattr(msg, 'content') and msg.content:
        print(f"Content: {msg.content}")
    
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"Tool Call: {tc['name']}({tc['args']})")

    print("-" * 60)

print("\n" + "=" * 60)
print("执行流程总结：")
print("1. HumanMessage: 用户输入")
print("2. AIMessage (with tool_calls): Agent决定调用工具")
print("3. ToolMessage: 工具执行结果")
print("4. AIMessage (final): Agent基于结果生成最终回答")

【Agent执行循环详解】

完整的消息历史：

步骤 1: HumanMessage
Content: 上海的天气如何？
------------------------------------------------------------

步骤 2: AIMessage
Tool Call: get_weather({'city': '上海'})
------------------------------------------------------------

步骤 3: ToolMessage
Content: 多云，温度18°C
------------------------------------------------------------

步骤 4: AIMessage
Content: 上海目前的天气是多云，温度18°C。需要了解其他城市的天气吗？
------------------------------------------------------------

执行流程总结：
1. HumanMessage: 用户输入
2. AIMessage (with tool_calls): Agent决定调用工具
3. ToolMessage: 工具执行结果
4. AIMessage (final): Agent基于结果生成最终回答


## 4. ReAct模式（Reasoning + Acting）

Agent遵循ReAct模式：交替进行推理和行动

In [7]:
print("【ReAct模式示例】")
print()

# 创建多个工具
@tool
def add(a: int, b: int) -> int:
    """计算两个数的和"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """计算两个数的乘积"""
    return a * b

# 创建Agent
math_agent = create_agent(
    model=model,
    tools=[add, multiply],
    system_prompt="你是数学助手。解决问题时，先思考需要做什么，再调用工具。"
)

# 复杂问题：需要多步推理
response = math_agent.invoke({
    "messages": [{"role": "user", "content": "计算 (3 + 5) * 2 的结果"}]
})

print("问题: 计算 (3 + 5) * 2 的结果")
print()
print("Agent的思考和行动过程：")
print("-" * 60)

step = 1
for msg in response['messages']:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tc in msg.tool_calls:
            print(f"步骤 {step} [行动]: 调用 {tc['name']}({tc['args']})")
            step += 1
    elif msg.__class__.__name__ == 'ToolMessage':
        print(f"步骤 {step} [观察]: 结果 = {msg.content}")
        step += 1

print(f"\n最终答案: {response['messages'][-1].content}")

【ReAct模式示例】

问题: 计算 (3 + 5) * 2 的结果

Agent的思考和行动过程：
------------------------------------------------------------
步骤 1 [行动]: 调用 add({'a': 3, 'b': 5})
步骤 2 [行动]: 调用 multiply({'a': 8, 'b': 2})
步骤 3 [观察]: 结果 = 8
步骤 4 [观察]: 结果 = 16

最终答案: 首先计算括号内的加法 3 + 5 = 8，然后将结果乘以 2，即 8 * 2 = 16。所以 (3 + 5) * 2 的结果是 16。


## 5. system_prompt的重要性

system_prompt指导Agent的行为和决策

In [8]:
print("【system_prompt对比】")
print()

# 创建搜索工具
@tool
def search_web(query: str) -> str:
    """在网上搜索信息"""
    return f"搜索'{query}'的结果：找到相关信息..."

# Agent 1: 没有明确指导
agent1 = create_agent(
    model=model,
    tools=[search_web],
    system_prompt="你是助手"
)

# Agent 2: 有明确指导
agent2 = create_agent(
    model=model,
    tools=[search_web],
    system_prompt="""你是信息助手。

使用指南：
- 对于需要实时信息的问题，使用search_web工具
- 对于常识问题，直接回答
- 搜索前先判断是否真的需要搜索
"""
)

# 测试相同问题
question = "Python是什么？"

print(f"问题: {question}")
print()

response1 = agent1.invoke({"messages": [{"role": "user", "content": question}]})
response2 = agent2.invoke({"messages": [{"role": "user", "content": question}]})

# 检查是否调用了工具
used_tool1 = any(hasattr(msg, 'tool_calls') and msg.tool_calls for msg in response1['messages'])
used_tool2 = any(hasattr(msg, 'tool_calls') and msg.tool_calls for msg in response2['messages'])

print(f"Agent1 (简单prompt) 是否调用工具: {used_tool1}")
print(f"Agent2 (详细prompt) 是否调用工具: {used_tool2}")
print()
print("结论: 好的system_prompt能帮助Agent做出更合理的决策")

【system_prompt对比】

问题: Python是什么？

Agent1 (简单prompt) 是否调用工具: True
Agent2 (详细prompt) 是否调用工具: False

结论: 好的system_prompt能帮助Agent做出更合理的决策


## 6. Agent的底层架构

Agent基于LangGraph构建，但你不需要了解LangGraph就能使用

In [9]:
print("【Agent的底层架构】")
print()

print("create_agent() 返回的是什么？")
print(f"类型: {type(agent)}")
print(f"类名: {agent.__class__.__name__}")
print()

print("Agent的执行流程（Graph结构）：")
print("""
┌─────────────┐
│   START     │
└──────┬──────┘
       │
       ▼
┌─────────────┐
│    Model    │ ← 调用LLM，决定是否使用工具
└──────┬──────┘
       │
       ▼
    有tool_calls?
       │
    ┌──┴──┐
    │ 是  │ 否
    │     │
    ▼     ▼
┌────────┐  ┌────────┐
│ Tools  │  │  END   │
└───┬────┘  └────────┘
    │
    │ 执行工具
    │
    └──→ 回到Model
""")

print("关键点：")
print("1. create_agent() 自动构建了一个LangGraph")
print("2. Graph包含Model节点和Tools节点")
print("3. 自动处理循环逻辑")
print("4. 你只需调用invoke()，其余都是自动的")

【Agent的底层架构】

create_agent() 返回的是什么？
类型: <class 'langgraph.graph.state.CompiledStateGraph'>
类名: CompiledStateGraph

Agent的执行流程（Graph结构）：

┌─────────────┐
│   START     │
└──────┬──────┘
       │
       ▼
┌─────────────┐
│    Model    │ ← 调用LLM，决定是否使用工具
└──────┬──────┘
       │
       ▼
    有tool_calls?
       │
    ┌──┴──┐
    │ 是  │ 否
    │     │
    ▼     ▼
┌────────┐  ┌────────┐
│ Tools  │  │  END   │
└───┬────┘  └────────┘
    │
    │ 执行工具
    │
    └──→ 回到Model

关键点：
1. create_agent() 自动构建了一个LangGraph
2. Graph包含Model节点和Tools节点
3. 自动处理循环逻辑
4. 你只需调用invoke()，其余都是自动的


## 7. Agent vs 手动bind_tools

对比两种方式的区别

## 对比：Agent vs 手动bind_tools

### 方式1：手动bind_tools（需要自己写循环）

```python
model_with_tools = model.bind_tools([tool])
messages = [{"role": "user", "content": "问题"}]

# 需要手动循环
for _ in range(max_iterations):
    response = model_with_tools.invoke(messages)
    messages.append(response)
    
    if not response.tool_calls:
        break  # 手动判断停止
    
    # 手动执行工具
    for tool_call in response.tool_calls:
        result = tool.invoke(tool_call)
        messages.append(result)
```

---

### 方式2：使用Agent（推荐）

```python
agent = create_agent(model=model, tools=[tool])

# 一行搞定，自动循环
response = agent.invoke({"messages": [...]})
```

---

### 对比总结

| 特性       | bind_tools  | create_agent |
|------------|-------------|-------------|
| 代码量     | 多（20+行） | 少（3-5行） |
| 循环处理   | 手动        | 自动        |
| 工具执行   | 手动        | 自动        |
| 错误处理   | 手动        | 自动        |
| 适用场景   | 学习/简单场景| 生产环境   |
| 灵活性     | 高          | 中          |
| 易用性     | 低          | 高          |

## 8. 实战项目：智能助手Agent

构建一个具备多种能力的智能助手

In [11]:
print("【智能助手Agent】")
print()

from datetime import datetime

# 定义多个工具
@tool
def get_current_time() -> str:
    """获取当前时间"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def calculate(expression: str) -> str:
    """计算数学表达式，例如: '2+3*4'"""
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"计算错误: {str(e)}"

@tool
def translate_to_english(text: str) -> str:
    """将中文翻译成英文（模拟）"""
    translations = {
        "你好": "Hello",
        "谢谢": "Thank you",
        "再见": "Goodbye"
    }
    return translations.get(text, f"Translation of '{text}'")

# 创建智能助手
assistant = create_agent(
    model=model,
    tools=[get_current_time, calculate, translate_to_english],
    system_prompt="""你是一个智能助手，可以：
1. 查询当前时间
2. 进行数学计算
3. 翻译中文到英文

根据用户需求选择合适的工具。如果不需要工具，直接回答。"""
)

print("智能助手已创建！")
print(f"可用工具: {[t.name for t in [get_current_time, calculate, translate_to_english]]}")

【智能助手Agent】

智能助手已创建！
可用工具: ['get_current_time', 'calculate', 'translate_to_english']


### 8.1 测试智能助手

In [12]:
print("【测试智能助手】")
print()

# 测试不同类型的请求
test_queries = [
    "现在几点了？",
    "计算 15 * 8 + 20",
    "把'你好'翻译成英文",
    "你是谁？"  # 不需要工具
]

for query in test_queries:
    print(f"用户: {query}")
    response = assistant.invoke({"messages": [{"role": "user", "content": query}]})
    
    # 检查使用了哪些工具
    tools_used = []
    for msg in response['messages']:
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            tools_used.extend([tc['name'] for tc in msg.tool_calls])
    
    print(f"使用工具: {tools_used if tools_used else '无'}")
    print(f"回答: {response['messages'][-1].content}")
    print("-" * 60)
    print()

【测试智能助手】

用户: 现在几点了？
使用工具: ['get_current_time']
回答: 现在是2025年12月20日，16点44分。还有什么我可以帮您的吗？
------------------------------------------------------------

用户: 计算 15 * 8 + 20
使用工具: ['calculate']
回答: 15 * 8 + 20 的计算结果是 140。
------------------------------------------------------------

用户: 把'你好'翻译成英文
使用工具: ['translate_to_english']
回答: "你好" 翻译成英文是 "Hello"。
------------------------------------------------------------

用户: 你是谁？
使用工具: 无
回答: 我是你的智能助手，可以帮助你查询当前时间、进行数学计算和翻译中文到英文。有什么我可以帮你的吗？
------------------------------------------------------------



## 9. 总结与最佳实践

### 核心概念回顾

1. **Agent = LLM + Tools + 自动循环**
2. **ReAct模式**：Reasoning（推理）+ Acting（行动）交替进行
3. **create_agent()**：LangChain v1的标准Agent API
4. **自动化**：Agent自动处理工具调用循环，无需手动管理
5. **基于LangGraph**：底层使用LangGraph，但使用时无需了解细节

### Agent的关键组成

1. **model**：语言模型（可以是字符串或模型实例）
2. **tools**：工具列表（使用@tool装饰器创建）
3. **system_prompt**：指导Agent行为的系统提示词
4. **middleware**：可选的中间件（错误处理、日志等）

### 最佳实践

**工具设计：**
- 工具功能要单一明确
- 工具描述要清晰准确
- 参数类型要明确（使用类型提示）
- 避免工具过多（建议 < 10个）

**system_prompt设计：**
- 明确Agent的角色和能力
- 说明何时使用哪个工具
- 提供使用示例和指导原则
- 设置必要的限制和规则

**执行控制：**
- 设置合理的最大迭代次数
- 监控工具调用次数
- 处理工具执行失败的情况
- 记录Agent的决策过程

**性能优化：**
- 减少不必要的工具调用
- 缓存常见查询结果
- 使用更快的模型（如果可以）
- 批量处理多个请求

### 常见问题与解决

**问题1: Agent不调用工具**
- 检查工具描述是否清晰
- 检查system_prompt是否指导明确
- 确认问题确实需要工具

**问题2: Agent调用错误的工具**
- 改进工具描述的准确性
- 在system_prompt中明确工具用途
- 减少功能相似的工具数量

**问题3: Agent陷入循环**
- 设置最大迭代次数
- 改进system_prompt，明确停止条件
- 检查工具返回是否有用

**问题4: 响应太慢**
- 减少工具数量
- 优化工具执行速度
- 使用更快的模型
- 考虑异步执行

**问题5: 成本太高**
- 监控LLM调用次数
- 优化system_prompt减少无效调用
- 缓存常见请求
- 使用更便宜的模型

### Agent vs bind_tools 选择指南

| 场景 | 推荐方式 |
|------|---------|
| 学习Tool Calling原理 | bind_tools |
| 简单的一次性工具调用 | bind_tools |
| 生产环境 | create_agent |
| 需要多次工具调用 | create_agent |
| 复杂推理任务 | create_agent |
| 需要错误处理 | create_agent |

### 下一步学习

- **Agent with Tools**：深入学习工具管理和错误处理
- **Agent Memory**：为Agent添加记忆能力
- **Multi-Agent**：多个Agent协作
- **LangGraph**：自定义Agent流程（高级）